# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
%matplotlib inline

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [36]:
data = pd.read_csv("output/weather_dataframe.csv")
weather_data = data.drop(columns=["Unnamed: 0"], axis=1)
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed
0,vostok,76,RU,1580015416,80,46.49,135.88,7.09,7.49
1,poplar bluff,1,US,1580015818,75,36.76,-90.39,41.00,1.30
2,new norfolk,75,AU,1580015930,30,-42.78,147.06,69.80,21.92
3,hervey bay,40,AU,1580015930,43,-25.30,152.85,88.00,11.41
4,high level,90,CA,1580015931,100,58.52,-117.14,17.60,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [37]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [75]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat.", "Lng."]]
humidity = weather_data["Humidity"].astype(int)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
# Create a copy of weather data
narrowed_city_df = weather_data.copy()

# User chosen conditions
selectMinTemp = int(input("What's the lowest temperature you'd like to experience? "))
selectMaxTemp = int(input("What's the highest temperature you'd like to experience? "))
selectWind = int(input("How windy from 1-15? "))

# Apply filters
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] >= selectMinTemp]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Max Temp'] <= selectMaxTemp]  
narrowed_city_df = narrowed_city_df[narrowed_city_df['Wind Speed'] <= selectWind]
narrowed_city_df = narrowed_city_df[narrowed_city_df['Cloudiness'] == 0]

# Drop null values
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df = narrowed_city_df.reset_index()

narrowed_city_df.head()

What's the lowest temperature you'd like to experience? 60
What's the highest temperature you'd like to experience? 80
How windy from 1-15?  15


,index,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed
0,11,nohar,0,IN,1580015931,34,29.18,74.77,68.43,1.86
1,16,mar del plata,0,AR,1580015719,76,-38.00,-57.56,64.00,1.32
2,129,chongwe,0,ZM,1580015946,93,-15.33,28.68,60.80,1.12
3,144,arraial do cabo,0,BR,1580015948,83,-22.97,-42.03,75.20,9.17
4,162,acajutla,0,SV,1580015950,89,13.59,-89.83,69.01,1.01


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
# Store into variable named hotel_df.
hotel_df = narrowed_city_df

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = "TBD"

hotels = []

x = 0

# for loop to search through city coords
for cities in hotel_df['City']:
    coordinates = str(hotel_df['Lat.'][x])+' , '+str(hotel_df['Lng.'][x])
    x+=1
    
    # Set parameters to search for hotels with 5000 meters.
    target_radius = 5000
    
    target_type = 'lodging'
    params = {
    "location": coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        
        # Store the first Hotel result into the DataFrame.
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append("TBD")

# Append hotel names to df
hotel_df['Hotel Name'] = hotels
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Lat.,Lng.,Max Temp,Wind Speed,Hotel Name
0,11,nohar,0,IN,1580015931,34,29.18,74.77,68.43,1.86,Hotel Eva Inn
1,16,mar del plata,0,AR,1580015719,76,-38.00,-57.56,64.00,1.32,Gran Hotel Mar del Plata habitaciones
2,129,chongwe,0,ZM,1580015946,93,-15.33,28.68,60.80,1.12,Farm KS
3,144,arraial do cabo,0,BR,1580015948,83,-22.97,-42.03,75.20,9.17,Pousada Porto Praia
4,162,acajutla,0,SV,1580015950,89,13.59,-89.83,69.01,1.01,Villavela Hotel Boutique
5,173,dwarka,0,IN,1580015867,72,22.24,68.97,72.64,8.23,Hotel Narayan Avezika Comfort
6,192,karimpur,0,IN,1580015954,35,23.97,88.62,70.38,7.11,gabrudanga jumma masjid
7,194,guerrero negro,0,MX,1580015766,80,27.98,-114.06,60.73,4.47,"Casa Laguna, Bed & Breakfast"
8,240,contai,0,IN,1580015960,35,21.78,87.75,74.12,5.01,Hotel Hindusthan
9,368,tezu,0,IN,1580015976,30,27.92,96.17,67.91,3.89,Circuit House


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat.", "Lng."]]

In [73]:
# Add marker layer ontop of heat map

# Optional: Mouseover
info_box_template2 = '''
Hotel: {Hotel Name}
City: {City}
Country: {Country}

'''

hotel_info2 = [info_box_template2.format(**row) for index, row in hotel_df.iterrows()]

markers = gmaps.marker_layer(locations, info_box_content=hotel_info, hover_text=hotel_info2)
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))